<a href="https://colab.research.google.com/github/YianKim/2022_uncertainty_aware_semisupervise/blob/main/Keras_UPS_SVHN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 32.2 MB/s 


In [ ]:
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import clone_model

import PIL
from PIL import Image

import pickle
import random
from tqdm import tqdm
from collections import Counter

from keras.layers.core import Lambda
from keras import backend as K

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import BatchNormalization
from keras.regularizers import l2
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import MaxPooling2D, AveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Reshape
from keras import optimizers
from keras.callbacks import *
from sklearn.metrics import *
from keras.models import load_model
import tensorflow_addons as tfa

from torchvision import transforms

import tensorflow as tf
import tensorflow.keras.backend as backend
import math
import gc

In [ ]:
def dummy_labels(labels):
  zero_labels = np.zeros([labels.shape[0], 10], np.int8)  
  for i in range(labels.shape[0]):
    zero_labels[i][labels[i]] = 1
  return(zero_labels)

# SVHN

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from scipy.io import loadmat
train_raw = loadmat('/content/drive/MyDrive/SVHN/train_32x32.mat')
test_raw = loadmat('/content/drive/MyDrive/SVHN/test_32x32.mat')

In [ ]:
train_images = train_raw['X']
train_labels = train_raw['y']

test_images = test_raw['X']
test_labels = dummy_labels(test_raw['y']-1)

train_images = train_images.swapaxes(2,3).swapaxes(1,2).swapaxes(0,1)
test_images = test_images.swapaxes(2,3).swapaxes(1,2).swapaxes(0,1)

In [ ]:
temp = [0,0,0,0,0,0,0,0,0,0]
label_indx = []
unlabel_indx = []

for i in range(73257) :
  if temp[(train_labels).reshape([-1])[i]-1] < 25 :
    temp[(train_labels).reshape([-1])[i]-1] += 1
    label_indx.append(i)
  else :
    unlabel_indx.append(i)

In [ ]:
lbl_train_images = train_images[label_indx]
lbl_train_labels = dummy_labels(train_labels[label_indx]-1)

In [ ]:
ubl_train_images = train_images[unlabel_indx]
ubl_train_labels = dummy_labels(train_labels[unlabel_indx]-1)

# pseudo labeling

In [ ]:
def basic_augmentation(imagearray):
  image = Image.fromarray(imagearray)
  
  tr2 = transforms.RandomRotation(15)
  tr3 = transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2)
  image = tr2(tr3(image))
  return(np.array(image))

def makeaugs(n, input):
  augs = []
  for j in range(n):
    for i in input:
      augs.append(basic_augmentation(np.array(i*255, np.uint8)))
  return(np.array(augs)/255)

### Mixup Augmentation


In [ ]:
def sample_beta_distribution(size, concentration_0=0.2, concentration_1=0.2):
    gamma_1_sample = tf.random.gamma(shape=[size], alpha=concentration_1)
    gamma_2_sample = tf.random.gamma(shape=[size], alpha=concentration_0)
    return gamma_1_sample / (gamma_1_sample + gamma_2_sample)

def mixup (size, data, alpha = 0.2):
  image, label = data
  L = sample_beta_distribution(size, alpha, alpha)
  XL = tf.reshape(L, (size, 1, 1, 1))
  YL = tf.reshape(L, (size, 1))
  IND1 = np.random.choice(len(label), size)
  IND2 = np.random.choice(len(label), size)
  newimage = XL*image[IND1] + (1-XL)*image[IND2]
  newlabel = YL*label[IND1] + (1-YL)*label[IND2]
  return (newimage, newlabel)

### 스케줄러

In [ ]:
class SGDR(Callback):

    def __init__(self, min_lr=0.0, max_lr=0.03, base_epochs=20, mul_epochs=2):
        super(SGDR, self).__init__()

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.base_epochs = base_epochs
        self.mul_epochs = mul_epochs

        self.cycles = 0.
        self.cycle_iterations = 0.
        self.trn_iterations = 0.

        self._reset()

    def _reset(self, new_min_lr=None, new_max_lr=None,
               new_base_epochs=None, new_mul_epochs=None):
        """Resets cycle iterations."""
        
        if new_min_lr != None:
            self.min_lr = new_min_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_base_epochs != None:
            self.base_epochs = new_base_epochs
        if new_mul_epochs != None:
            self.mul_epochs = new_mul_epochs
        self.cycles = 0.
        self.cycle_iterations = 0.
        
    def sgdr(self):
        
        cycle_epochs = self.base_epochs * (self.mul_epochs ** self.cycles)
        tide = ((self.cycles == 0) * 1) * (self.cycle_iterations*self.max_lr + (self.base_epochs - self.cycle_iterations)*self.min_lr) / self.base_epochs + ((self.cycles != 0) * 1)*(self.min_lr + 0.5 * (self.max_lr - self.min_lr) * (1 + np.cos(np.pi * (self.cycle_iterations + 1) / cycle_epochs)))
        return tide
        
    def on_train_begin(self, logs=None):
        
        if self.cycle_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.max_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.sgdr())
            
    def on_epoch_end(self, epoch, logs=None):
        
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)
        
        self.trn_iterations += 1
        self.cycle_iterations += 1
        if self.cycle_iterations >= self.base_epochs * (self.mul_epochs ** self.cycles):
            self.cycles += 1
            self.cycle_iterations = 0
            K.set_value(self.model.optimizer.lr, self.max_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.sgdr())

### main

In [ ]:
def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

def create_cnn_13():
  inputlayer = keras.Input(shape=(32, 32, 3))

  conv1a = Conv2D(64, (5,5), padding = 'same')
  bn1a = BatchNormalization()
  conv1b = Conv2D(64, (5,5), padding = 'same')
  bn1b = BatchNormalization()
  conv1c = Conv2D(64, (5,5), padding = 'same')
  bn1c = BatchNormalization()
  pl1 = MaxPooling2D(2, 2)
  MCdrop1 = PermaDropout(0.5)

  fc1 = Dense(1024)
  fc2 = Dense(10)
  activ = keras.layers.LeakyReLU(0.1)

  model = Sequential([
                  inputlayer, 
                  tfa.layers.WeightNormalization(conv1a), bn1a, activ, pl1,
                  tfa.layers.WeightNormalization(conv1b), bn1b, activ, pl1,
                  tfa.layers.WeightNormalization(conv1c), bn1c, activ, pl1,
                  MCdrop1, Flatten(),

                  fc1, fc2
                  ])
  
  return model

def compile_cnn_13(model):

  opt = keras.optimizers.SGD(0.03, momentum=0.9)

  model.compile(
    optimizer = opt,
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
  )

  return model

def cnn_13():

  model = create_cnn_13()
  model = compile_cnn_13(model)

  return model

def fit_and_labeling_cnn_13(Epoch, Batch):

  X = lbl_train_images
  y = lbl_train_labels

  lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
  early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=30, mode='auto')
  sgdr = SGDR(min_lr=0.0, max_lr=0.03, base_epochs=20) #스케줄러
  
  # size = len(y) * 3
  # newimage, newlabel = mixup(size, (X, y))
  augimage, auglabel = makeaugs(3, X), np.concatenate((y,y,y))
  X = np.concatenate((X, augimage))
  y = np.concatenate((y, auglabel))
  # del newimage, newlabel

  model.fit(
      x=X,
      y=y,
      epochs=Epoch,
      verbose=0,
#       validation_data = (valids1, valids2),
      batch_size=Batch,
#       callbacks=[sgdr, early_stopper]
      callbacks=[sgdr]
  )
    
  model_test_eval(model, test_images, test_labels)
  T = 1

  for predsamples in (range(10)):
    if predsamples == 0 :
      predictions = np.array(tf.nn.softmax(model.predict(ubl_train_images)/T))
      predictions = predictions.reshape((1,) + predictions.shape)
    else:
      pred = np.array(tf.nn.softmax(model.predict(ubl_train_images)/T))
      pred = pred.reshape((1,) + pred.shape)
      predictions = np.concatenate((predictions, pred))

  return predictions

def model_test_eval(model, test_images, test_labels):
  T = 1
  pred = np.array(tf.nn.softmax(model.predict(test_images)/T))
  for i in range(1,10):
    pred += np.array(tf.nn.softmax(model.predict(test_images)))
  acc = (np.argmax(pred,axis=1) == np.argmax(test_labels,axis=1))*1
  acc = sum(acc)/len(acc)
  print("test set 성능 : " + str(acc))

In [ ]:
def label_selecting():
  K_conf = 0.9
  K_uncert = 0.05

  pseudo = np.argmax(np.mean(predictions, axis=0), axis=1)
  conf = np.max(np.mean(predictions, axis=0), axis=1)
  uncert = np.std(predictions, axis=0)
  uncert = np.array([uncert[i][pseudo[i]] for i in range(len(pseudo))])

  select_pseudo = (1*(conf > K_conf)) * (1*(uncert < K_uncert))

  labels = []
  for i in pseudo:
    temp = [0,0,0,0,0,0,0,0,0,0]
    temp[i] = 1
    labels.append(temp)
  pseudo = np.array(labels)
#   pseudo = np.mean(predictions, axis=0)

  lbl_idx = []
  ubl_idx = []
  k = 0
  for i in select_pseudo:
    if i == 1:
      lbl_idx.append(k)
    if i == 0:
      ubl_idx.append(k)
    k += 1

    
  ubl_append = ubl_train_images[lbl_idx]
  pseudo_append = pseudo[lbl_idx]
    
  if itr < 20:
      try: 
        numsamples = np.min(list(Counter(np.argmax(pseudo_append, axis=1)).values()))
      except:
        numsamples = 0
      multlabel = np.argmax(pseudo_append, axis=1)
      sufindx = random.sample(range(len(multlabel)), len(multlabel))

      idxcounter = [0,0,0,0,0,0,0,0,0,0]
      idxsample = []

      for i in sufindx:
#         if idxcounter[multlabel[i]] < numsamples+25:
        if idxcounter[multlabel[i]] < min(500, numsamples):
          idxcounter[multlabel[i]] += 1
          idxsample.append(i)
      
      image1 = np.concatenate((lbl_train_images, ubl_append[idxsample]))
      label1 = np.concatenate((lbl_train_labels, pseudo_append[idxsample]))
      image2 = np.concatenate((ubl_train_images[ubl_idx], ubl_append[np.delete(list(range(len(ubl_append))), idxsample)]))
  
  else:
      image1 = np.concatenate((lbl_train_images, ubl_append))
      label1 = np.concatenate((lbl_train_labels, pseudo_append))
      image2 = ubl_train_images[ubl_idx]

  return image1, label1, image2

In [ ]:
import time
start = time.time()

for itr in range(20):
  model = cnn_13()
  print(Counter(np.argmax(lbl_train_labels, axis=1)))
  predictions = fit_and_labeling_cnn_13(200, 64)
  lbl_train_images, lbl_train_labels, ubl_train_images = label_selecting()
  del predictions
#   teacher_model = model
  gc.collect()
  print("time :", time.time() - start)

print("time :", time.time() - start)
# CNN-13(update) + mixup augmentation(x4) + itr10balancing < 0.64
# CNN-13(update) + mixup augmentation(x8) + itr5balancing < 0.68?

Counter({0: 25, 8: 25, 1: 25, 2: 25, 4: 25, 7: 25, 6: 25, 3: 25, 5: 25, 9: 25})
test set 성능 : 0.4957744314689613
time : 181.35410523414612
Counter({0: 70, 8: 70, 1: 70, 2: 70, 4: 70, 7: 70, 6: 70, 3: 70, 5: 70, 9: 70})
test set 성능 : 0.5515135218192994
time : 442.6598792076111
Counter({0: 223, 8: 223, 1: 223, 2: 223, 4: 223, 7: 223, 6: 223, 3: 223, 5: 223, 9: 223})
test set 성능 : 0.5803242163491088
time : 950.1974520683289
Counter({0: 423, 8: 423, 1: 423, 2: 423, 4: 423, 7: 423, 6: 423, 3: 423, 5: 423, 9: 423})
test set 성능 : 0.6161263060848187
time : 1781.8189961910248
Counter({0: 690, 8: 690, 1: 690, 2: 690, 4: 690, 7: 690, 6: 690, 3: 690, 5: 690, 9: 690})
test set 성능 : 0.6354102642901045
time : 3076.934863805771
Counter({0: 817, 8: 817, 1: 817, 2: 817, 4: 817, 7: 817, 6: 817, 3: 817, 5: 817, 9: 817})
test set 성능 : 0.6324139520590043
time : 4560.2981333732605
Counter({0: 901, 8: 901, 1: 901, 2: 901, 4: 901, 7: 901, 6: 901, 3: 901, 5: 901, 9: 901})
test set 성능 : 0.6364858635525507
time :